<a href="https://colab.research.google.com/github/Patricio2088/Herramientas-IA-AIAA/blob/main/Pr%C3%A1ctica2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SE procede a instalar las librerias

In [1]:
!pip install pandas scikit-learn mlflow pyngrok flask

Autenticación de Ngrok (una vez)

In [2]:
!ngrok config add-authtoken 2xHpfarOCQJmV0CYnoUDqPvt6Oc_86V6mj11UvmcxpQVsSRwe


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
import pandas as pd
import mlflow
import mlflow.sklearn
import subprocess
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import joblib
from pyngrok import ngrok

# 1. Cargar los datos
url = "depression_data.csv"
df = pd.read_csv(url)  # Puedes reemplazarlo por tu CSV de Kaggle subido

# 2. Crear columna objetivo artificial: Depression
df["Depression"] = (
    (df["Sleep Patterns"] == "Poor") |
    (df["History of Mental Illness"] == "Yes") |
    (df["Family History of Depression"] == "Yes")
).astype(int)

# 3. Separar features y target
X = df.drop(columns=["Name", "Depression"])
y = df["Depression"]

# 4. Dividir
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# 5. Preprocesamiento
categorical = X.select_dtypes(include="object").columns.tolist()
numerical = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical),
    ("num", StandardScaler(), numerical)
])

# 6. Pipeline con modelo
pipeline = Pipeline([
    ("preprocessing", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=100, random_state=42))
])

# 7. Lanzar MLflow en segundo plano
mlflow_process = subprocess.Popen(
    ["mlflow", "server", "--backend-store-uri", "./mlruns", "--port", "9090"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.STDOUT,
)

print("✅ MLflow iniciado en el puerto 9090.")

# 8. Exponer MLflow
mlflow_url = ngrok.connect(9090)
print("🌐 MLflow disponible en:", mlflow_url)

# 9. Entrenar modelo con MLflow tracking
mlflow.set_tracking_uri("http://127.0.0.1:9090")
mlflow.set_experiment("Depression_Prediction")

with mlflow.start_run():
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", acc)
    mlflow.sklearn.log_model(pipeline, "model")

    # Guardar modelo localmente
    joblib.dump(pipeline, "model.pkl")
    print(f"✅ Modelo entrenado y guardado. Accuracy: {acc:.4f}")


✅ MLflow iniciado en el puerto 9090.
🌐 MLflow disponible en: NgrokTunnel: "https://e897-35-222-9-63.ngrok-free.app" -> "http://localhost:9090"


2025/05/18 22:49:31 INFO mlflow.tracking.fluent: Experiment with name 'Depression_Prediction' does not exist. Creating a new experiment.
2025/05/18 22:50:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Modelo entrenado y guardado. Accuracy: 1.0000
🏃 View run rare-mink-730 at: http://127.0.0.1:9090/#/experiments/910279813855682614/runs/254d57cffb07406098da7756add27c8d
🧪 View experiment at: http://127.0.0.1:9090/#/experiments/910279813855682614


Crear app.py

In [4]:
%%writefile app.py
from flask import Flask, request, render_template_string
import joblib
import pandas as pd

app = Flask(__name__)
model = joblib.load("model.pkl")

# Opciones para cada campo categórico
options = {
    "Marital Status": ["Single", "Married", "Divorced", "Widowed"],
    "Education Level": ["High School", "Associate Degree", "Bachelor's Degree", "Master's Degree", "PhD"],
    "Smoking Status": ["Smoker", "Former", "Non-smoker"],
    "Physical Activity Level": ["Sedentary", "Moderate", "Active"],
    "Employment Status": ["Employed", "Unemployed"],
    "Alcohol Consumption": ["Low", "Moderate", "High"],
    "Dietary Habits": ["Healthy", "Moderate", "Unhealthy"],
    "Sleep Patterns": ["Good", "Fair", "Poor"],
    "History of Mental Illness": ["Yes", "No"],
    "History of Substance Abuse": ["Yes", "No"],
    "Family History of Depression": ["Yes", "No"],
    "Chronic Medical Conditions": ["Yes", "No"]
}

form_html = """
<h2>Predicción de Depresión</h2>
<form method="post">
  Age: <input name="Age" type="number"><br>
  Number of Children: <input name="Number of Children" type="number"><br>
  Income: <input name="Income" type="number"><br>

  {% for field, choices in options.items() %}
    {{ field }}:
    <select name="{{ field }}">
      {% for choice in choices %}
        <option value="{{ choice }}">{{ choice }}</option>
      {% endfor %}
    </select><br>
  {% endfor %}

  <input type="submit" value="Predecir">
</form>

{% if prediction is not none %}
  <h3>¿Depresión?: {{ 'Sí' if prediction == 1 else 'No' }}</h3>
{% endif %}
"""

@app.route("/", methods=["GET", "POST"])
def predict():
    prediction = None
    if request.method == "POST":
        data = {k: [v] for k, v in request.form.items()}
        df = pd.DataFrame(data)
        prediction = model.predict(df)[0]
    return render_template_string(form_html, prediction=prediction, options=options)

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

Writing app.py


Ejecutar flask con ngrok

In [5]:
# Ejecutar app Flask
import subprocess
import time
from pyngrok import ngrok

# Lanzar Flask con subprocess
flask_process = subprocess.Popen(["python", "app.py"])

# Esperar unos segundos para asegurar que el servidor se inicie
time.sleep(5)

# Conectar ngrok al puerto 5000
flask_url = ngrok.connect(5000)
print("🌐 App Flask disponible en:", flask_url)


🌐 App Flask disponible en: NgrokTunnel: "https://7fd2-35-222-9-63.ngrok-free.app" -> "http://localhost:5000"
